In [1]:
import requests
from bs4 import BeautifulSoup

import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

from time import sleep
import pandas as pd
import numpy as np
import datetime
import pickle
import os

# Resolución captcha:
import sounddevice as sd
from scipy.io.wavfile import write
from openai import OpenAI
from dotenv import load_dotenv

# Probar get_text()

# 0. Funciones Captcha

In [2]:
def record_audio(duration=5, sample_rate=44100):
    print("Recording...")

    # Record audio
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=2, dtype=np.int16)
    sd.wait()

    print("Recording finished.")

    return audio_data

def save_audio(audio_data, file_path="recorded_audio.mp3", sample_rate=44100):
    print("Saving audio...")

    # Save audio to file using scipy.io.wavfile.write
    write(file_path, sample_rate, audio_data)

    print(f"Audio saved to {file_path}")

# 1. Infoempleo Scraper

## 1.1. URL ofertas de empleo

In [3]:
def url_scraper_infoempleo(url, opts, nuevas= True):
    
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    
    browser.get(url)
    sleep(2)
    
    # Acepto cookies
    browser.find_element(By.ID, "onetrust-accept-btn-handler").click()
    sleep(1)
    
    url_empleos = []
    while True:

        try:
            browser.find_element(By.ID, "lightbox").find_element(By.CLASS_NAME, "close").click()
            sleep(1)
        except:
            pass
        
        if nuevas:
            # Boton ofertas en las últimas 24h:
            browser.find_element(By.ID, "fechapublicacion1").click()
            sleep(1)

        soup = BeautifulSoup(browser.page_source, "html.parser")
        empleos = soup.find("div", class_= "main-content").find_all("li", class_= "offerblock")

        for url in empleos:
            try:
                url_oferta = "https://www.infoempleo.com" + url.find("a")["href"]
                url_empleos.append(url_oferta)
            except:
                pass

        try:
            # Hago scroll hasta el final:
            elemento_objetivo = browser.find_elements(By.CLASS_NAME, "related-offer-item")[-1]
            browser.execute_script("arguments[0].scrollIntoView(true);", elemento_objetivo)
            sleep(1)
            # Siguiente página:
            browser.find_element(By.CLASS_NAME, "pagination").find_element(By.CLASS_NAME, "next").click()
            sleep(2)
        except:
            break
            
    return url_empleos

## 1.2. Información ofertas de empleo

In [4]:
def scraper_infoempleo(url_empleos, opts):
    # Inicio browser:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    
    # Valores fijos:
    portal = "infoempleo"
    fecha_scrapeo = datetime.datetime.now().date()
    
    
    datos_ofertas_empleo = []
    contador = 0
    for empleo in url_empleos:
    
        browser.get(empleo)
        sleep(2)

        try:
            # Acepto cookies
            browser.find_element(By.ID, "onetrust-accept-btn-handler").click()
        except:
            pass

        # Cierro popup si aparece:
        try:
            browser.find_element(By.ID, "lightbox").find_element(By.CLASS_NAME, "close").click()
            sleep(1)
        except:
            pass

        soup_oferta = BeautifulSoup(browser.page_source, "html.parser")

        try:
            titulo = soup_oferta.find("div", class_= "title-wrapper").find("h1").text

        except:
            titulo = np.nan

        try:
            empresa = soup_oferta.find("div", class_= "title-wrapper").find("li", class_= "companyname").text

        except:
            empresa = np.nan

        try:
            presencialidad = soup_oferta.find("div", class_= "title-wrapper").find("li", class_= "badge").text

        except:
            presencialidad = np.nan    

        try:
            fecha = soup_oferta.find("div", class_= "title-wrapper").find("li", class_= "mt10").text.strip()

        except:
            fecha = np.nan

        try:
            ubicacion = soup_oferta.find("div", class_= "title-wrapper").find("li", class_= "block").text.strip()

        except:
            ubicacion = np.nan

        # Bloque de características:

        try:
            browser.find_element(By.CLASS_NAME, "areapos-vmore").click()
        except:
            pass

        bullet_points = soup_oferta.find("div", class_= "offer-excerpt").find_all("ul", class_= "inline")

        try:
            experiencia = bullet_points[0].find_all("p")[0].text
        except:
            experiencia = np.nan

        try:
            salario = bullet_points[0].find_all("p")[1].text
        except:
            salario = np.nan

        try:
            funciones = [funcion.text for funcion in bullet_points[1].find_all("li")[1:]]
        except:
            funciones = np.nan

        try:
            solicitudes = bullet_points[2].find_all("p")[1].text
        except:
            solicitudes = np.nan        

        try:
            tipo_contrato = bullet_points[3].find_all("p")[0].text
        except:
            tipo_contrato = np.nan

        try:
            jornada = bullet_points[3].find_all("p")[1].text
        except:
            jornada = np.nan  

        cuerpo_oferta = soup_oferta.find("div", class_= "offer").find_all("pre")

        try:
            descripcion = cuerpo_oferta[0].text.replace("\n", "").replace("*", "").strip()
        except:
            descripcion = np.nan    

        try:
            herramientas = cuerpo_oferta[1].text.replace("\n", "").replace("*", "").strip()
        except:
            herramientas = np.nan

        datos_ofertas_empleo.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, presencialidad, funciones, jornada, experiencia, tipo_contrato, salario, solicitudes, fecha_scrapeo, empleo, portal])
        contador += 1

        if contador % 100 == 0:
            sleep(20)
            
            
    df = pd.DataFrame(datos_ofertas_empleo, columns= ["titulo", "empresa", "fecha", "herramientas", "descripcion", "ubicacion", "presencialidad", "funciones", "jornada", "experiencia", "tipo_contrato", "salario", "solicitudes", "fecha_scrapeo", "url", "portal"])
    
    return df 

# 2. Talenthacker Scraper

## 2.1. URL ofertas de empleo

In [5]:
def url_scraper_talenthacker(url, opts):
    
    # Inicio navegador
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()    
    browser.get(url)
    sleep(2)
    
    # Acepto cookies:
    browser.find_element(By.CLASS_NAME, "q-banner__actions").find_element(By.CLASS_NAME, "buttons-wrapper").find_element(By.CLASS_NAME, "q-btn--standard").click()

    # Hago scroll hasta poder ver todas las ofertas disponibles:
    last_height = browser.execute_script("return document.documentElement.scrollHeight")
    while True:
        # Scroll:
        browser.execute_script("window.scrollTo(0,document.documentElement.scrollHeight);")
        sleep(3)
        # Calcula la altura del nuevo scroll y la compara con la del último:
        new_height = browser.execute_script("return document.documentElement.scrollHeight")

        if new_height == last_height:
            print("Ya no hay más página.")
            break
        last_height = new_height

    soup = BeautifulSoup(browser.page_source, "html.parser")

    talent_hacker = "https://talenthackers.net"
    ofertas = soup.find("div", class_= "jobs-list-wrapper").find_all("div", class_= "col-12")

    url_ofertas = []
    for oferta in ofertas:
        url_oferta = talent_hacker + oferta.find("a")["href"]
        url_ofertas.append(url_oferta)
        
    return url_ofertas

## 2.2. Información ofertas de empleo

In [6]:
def scraper_talenthacker(url_empleos, opts, bullet_point):
    # Inicio browser:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    
    # Valores fijos:
    portal = "talenthacker"
    fecha_scrapeo = datetime.datetime.now().date()
    
    
    datos_ofertas_empleo = []
    contador = 0
    for empleo in url_empleos:
        
        browser.get(empleo)
        sleep(2)
        
        try:
            # Acepto cookies
            browser.find_element(By.CLASS_NAME, "q-banner__actions").find_element(By.CLASS_NAME, "buttons-wrapper").find_element(By.CLASS_NAME, "q-btn--standard").click()
        except:
            pass
        
        soup_oferta = BeautifulSoup(browser.page_source, "html.parser")
        
        #########################  INFORMACION  ##########################################################################
        
        try:
            titulo = soup_oferta.find("span", class_= "spot-title").text
        except:
            titulo = np.nan
            
        try:
            herramientas = [herramienta.text for herramienta in soup_oferta.find("div", class_= "spot-skills").find_all("a")]
        except:
            herramientas = np.nan
            
        try:
            descripcion = soup_oferta.find_all("div", class_= "block full-width text-body2")[0].text
        except:
            descripcion = np.nan
            
        try:
            funciones = soup_oferta.find_all("div", class_= "block full-width text-body2")[1].text
        except:
            funciones = np.nan
            
            
        imagenes = soup_oferta.find_all("div", class_= "th-body-md flex no-wrap q-ma-sm")
        for imagen in imagenes:
            img = imagen.find("img")["src"]
            
            if img == bullet_point["ubicacion"]:
                try:
                    ubicacion = imagen.text
                except:
                    ubicacion = np.nan
                    
            elif img == bullet_point["presencialidad"]:
                try:
                    presencialidad = imagen.text.strip()
                except:
                    presencialidad = np.nan
                    
            elif img == bullet_point["jornada_tipo"]:
                try:
                    tipo_contrato = imagen.text.split("·")[0].strip()
                    jornada = imagen.text.split("·")[1].strip()
                except:
                    tipo_contrato = np.nan 
                    jornada = np.nan
            
            if img == bullet_point["experiencia"]:
                try:
                    experiencia = imagen.text.strip()
                except:
                    experiencia = np.nan

            
            elif img == bullet_point["salario"]:
                try:
                    salario = imagen.text.strip()
                except:
                    salario = np.nan

        
        ###################################################################################################################
        datos_ofertas_empleo.append([titulo, herramientas, descripcion, ubicacion, presencialidad, funciones, jornada, experiencia, tipo_contrato, salario, fecha_scrapeo, empleo, portal])
        contador += 1

        if contador % 100 == 0:
            sleep(20)
            
            
    df = pd.DataFrame(datos_ofertas_empleo, columns= ["titulo", "herramientas", "descripcion", "ubicacion", "presencialidad", "funciones", "jornada", "experiencia", "tipo_contrato", "salario", "fecha_scrapeo", "url", "portal"])
    
    return df 

# 3. Tecnoempleo Scraper

## 3.1. Número de páginas

In [7]:
def pag_counter(opts,limite=1000, nuevos= True):

    # Inicializo navegador:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    
    # Entro en tecnoempleo:
    if nuevos:
        url_paginas = f"https://www.tecnoempleo.com/ofertas-trabajo/?ult_24h=,1,&pagina={limite}"
        
    else:
        url_paginas = f"https://www.tecnoempleo.com/ofertas-trabajo/?pagina={limite}"
    
    browser.get(url_paginas)
    
    # Acepto cookies
    browser.find_elements(By.CLASS_NAME, "col-6")[0].click()
    
    # Saco el número de páginas
    soup = BeautifulSoup(browser.page_source, "html.parser")
    num_paginas = int(soup.find("li", class_= "active").text)
    
    return num_paginas    

## 3.2. Información ofertas de empleo

In [8]:
def scraper_tecnoempleo(num_paginas, opts, nuevos= True):
    
    portal = "tecnoempleo"
    fecha_scrapeo = datetime.datetime.now().date()
    
    # Inicializo navegador:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    
    datos_ofertas_empleo = []
    
    contador = 0
    for pagina in range(num_paginas + 1):
        
        if nuevos:
            tecno_url = f"https://www.tecnoempleo.com/ofertas-trabajo/?ult_24h=,1,&pagina={pagina}"
        else:
            tecno_url = f"https://www.tecnoempleo.com/ofertas-trabajo/?pagina={pagina}"

        browser.get(tecno_url)
        sleep(1)
        
        # Acepto cookies
        try:
            browser.find_elements(By.CLASS_NAME, "col-6")[0].click()
        except:
            pass

        # Saco las urls de cada puesto ofertado y saco la información
        soup_page = BeautifulSoup(browser.page_source, "html.parser")

        ofertas_empleo = soup_page.find_all("div", class_= "col-10")

        for oferta in ofertas_empleo:

            # URL oferta de empleo
            url_oferta = oferta.find("a")["href"]
            # Entro en la oferta
            browser.get(url_oferta)
            sleep(1)
            # Saco información relevante de la oferta
            soup_oferta = BeautifulSoup(browser.page_source, "html.parser")

            lista_li = soup_oferta.find("ul", class_= "fs--15").find_all("li", class_= "border-bottom")

            ubicacion, funciones, jornada, experiencia, tipo_contrato, salario = [np.nan] * 6
            for li in lista_li:

                tag = li.find("span", class_= "d-inline-block").text

                if tag == "Ubicación" : 
                    ubicacion = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Funciones":
                    funciones = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Jornada":
                    jornada = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Experiencia":
                    experiencia = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Tipo contrato":
                    tipo_contrato = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

                elif tag == "Salario":
                    salario = li.find("span", class_= "float-end").text.strip().replace("\xa0", "").replace("\t", "").replace("\n", "")

            titulo = soup_oferta.find("div", class_= "col-lg-8").find("h1").text.replace("\t", "").replace("Urgente\n", "").strip()
        
            try:
                empresa = soup_oferta.find("div", class_= "col-lg-8").find("a", class_= "fs--18").text.strip()
            except:
                empresa = np.nan

            fecha = soup_oferta.find("div", class_= "col-lg-8").find("span", "ml-4").text.strip().replace("Actualizada", "")

            herramientas = soup_oferta.find("ul", class_= "fs--15").find("li", class_= "mb-3").text.strip().replace("\n", ", ")

            descripcion = soup_oferta.find("div", class_= "mt-4").text.replace("\n", " ").replace("\t", " ").replace("Descripción de la oferta de empleo", "").strip()

            datos_ofertas_empleo.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, funciones, jornada, experiencia, tipo_contrato, salario, fecha_scrapeo, url_oferta, portal])
            
        contador =+ 1
        if contador % 2 == 0:
            sleep(3)
            
    df = pd.DataFrame(datos_ofertas_empleo, columns= ["titulo", "empresa", "fecha", "herramientas", "descripcion", "ubicacion", "funciones", "jornada", "experiencia", "tipo_contrato", "salario", "fecha_scrapeo", "url", "portal"])
    
    return df

# 4. Infojobs Scraper

## 4.1. URL ofertas de empleo

In [9]:
def url_scraper_infojobs(opts, api_key, nuevas= True):
    pagina = 1
    
    if nuevas:
        url_infojobs = f"https://www.infojobs.net/jobsearch/search-results/list.xhtml?keyword=&categoryIds=150&segmentId=&page={pagina}&sortBy=PUBLICATION_DATE&onlyForeignCountry=false&sinceDate=_24_HOURS"
    
    else:
        url_infojobs = f"https://www.infojobs.net/ofertas-trabajo/informatica-telecomunicaciones?keyword=&categoryIds=150&segmentId=&page={pagina}&sortBy=PUBLICATION_DATE&onlyForeignCountry=false&sinceDate=ANY"
    
    # Initialize:
    browser = webdriver.Chrome(options=opts)
    browser.maximize_window()
    browser.get(url_infojobs)
    sleep(4)
    browser.get(url_infojobs)
    sleep(4)

    ##################################################Captcha##################################################

    browser.find_element(By.CLASS_NAME, "geetest_radar_tip").click()
    sleep(2)
    browser.find_element(By.CLASS_NAME, "geetest_voice").click()
    sleep(2)
    browser.find_element(By.CLASS_NAME, "geetest_replay").click()
    audio_data = record_audio(duration=15)
    sleep(2)
    save_audio(audio_data)
    sleep(2)

    client = OpenAI(api_key= api_key)
    audio_file = os.getcwd() + "\\recorded_audio.mp3"

    audio_file= open(audio_file, "rb")
    transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file)

    codigo = transcript.text.replace("Introduzca lo que oiga.", "").replace(", ", "").replace(".", "").strip()

    barra = browser.find_element(By.CLASS_NAME, "geetest_input")
    barra.send_keys(codigo)
    sleep(2)
    browser.find_element(By.CLASS_NAME, "geetest_box").find_element(By.CLASS_NAME, "geetest_btn").click()
    sleep(5)
    ###########################################################################################################

    # Acepto cookies
    browser.find_elements(By.ID, "didomi-notice-agree-button")[0].click()
    sleep(1)
    
    url_empleos = []
    while True:

        # Saco url empleos:
        soup_urls = BeautifulSoup(browser.page_source, "html.parser")

        try:
            urls = soup_urls.find_all("div", class_= "ij-OfferCardContent-description")
        except:
            break

        for url in urls:
            oferta_url = url.find("h2").find("a")["href"].replace("//", "")
            url_empleos.append(oferta_url)

        pagina += 1
        browser.find_elements(By.CLASS_NAME, "sui-MoleculePagination-item")[-1].click()
        sleep(2)

        lim_paginas = len(browser.find_elements(By.CLASS_NAME, "sui-MoleculePagination-item"))

        if pagina > 1:
            if lim_paginas < 7:
                break
                
    url_empleos_final = ["https://" + empleo for empleo in url_empleos]
    
    return url_empleos_final

## 4.2. Información ofertas de empleo

In [1]:
def scraper_infojobs(url_empleos_final, opts, api_key):
    datos_ofertas_empleo = []
    contador = 0
    
    portal = "infojobs"
    fecha_scrapeo = datetime.datetime.now().date()

    for empleo in url_empleos_final[contador:]:

        if contador == 0:
            browser = webdriver.Chrome(options=opts)
            browser.maximize_window()
            browser.get(empleo)
            sleep(4)

            ##########Captcha##########
            browser.find_element(By.CLASS_NAME, "geetest_radar_tip").click()
            sleep(2)
            browser.find_element(By.CLASS_NAME, "geetest_voice").click()
            sleep(2)
            browser.find_element(By.CLASS_NAME, "geetest_replay").click()
            audio_data = record_audio(duration=15)
            sleep(2)
            save_audio(audio_data)

            client = OpenAI(api_key= api_key)
            audio_file = os.getcwd() + "\\recorded_audio.mp3"

            audio_file= open(audio_file, "rb")
            transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file)

            codigo = transcript.text.replace("Introduzca lo que oiga.", "").replace(", ", "").replace(".", "").strip()

            barra = browser.find_element(By.CLASS_NAME, "geetest_input")
            barra.send_keys(codigo)
            sleep(2)
            browser.find_element(By.CLASS_NAME, "geetest_box").find_element(By.CLASS_NAME, "geetest_btn").click()
            sleep(5)
            ###########################

            # Acepto cookies
            browser.find_elements(By.ID, "didomi-notice-agree-button")[0].click()
            sleep(1)

        else:
            browser.get(empleo)
            sleep(3)

        # Saco información relevante de la oferta:    
        soup_oferta = BeautifulSoup(browser.page_source, "html.parser")

        try:
            captcha = soup_oferta.find("div", attrs= attrs_ad).text        
            if captcha == "Hacer clic para comprobarReintentar":
                ##########Captcha##########
                browser.find_element(By.CLASS_NAME, "geetest_radar_tip").click()
                sleep(2)
                browser.find_element(By.CLASS_NAME, "geetest_voice").click()
                sleep(2)
                browser.find_element(By.CLASS_NAME, "geetest_replay").click()
                audio_data = record_audio(duration=15)
                sleep(2)
                save_audio(audio_data)

                client = OpenAI(api_key= api_key)
                audio_file = os.getcwd() + "\\recorded_audio.mp3"

                audio_file= open(audio_file, "rb")
                transcript = client.audio.transcriptions.create(model="whisper-1", file=audio_file)

                codigo = transcript.text.replace("Introduzca lo que oiga.", "").replace(", ", "").replace(".", "").strip()

                barra = browser.find_element(By.CLASS_NAME, "geetest_input")
                barra.send_keys(codigo)
                sleep(2)
                browser.find_element(By.CLASS_NAME, "geetest_box").find_element(By.CLASS_NAME, "geetest_btn").click()
                sleep(5)
                ###########################
                
        except:
            pass

        try:
            attrs_tit= {"id": "prefijoPuesto"}
            titulo = soup_oferta.find("div", class_= "heading-addons").find("h1", attrs= attrs_tit).text
        except:
            titulo = np.nan
        try:
            empresa = soup_oferta.find("div", class_= "heading-addons").find("a", class_= "link").text
        except:
            empresa = np.nan
        try:
            fecha = soup_oferta.find("div", class_= "row-matrioska").find("span", class_= "marked").text
        except:
            try:
                lista_bullets = soup_oferta.find("div", class_= "row-matrioska").find("ul", class_= "list-bullet-default").find_all("li")
                for bullet in lista_bullets:
                    tag_bullet = bullet.text.strip()[:9]
                    if tag_bullet == "Publicada":
                        fecha = bullet.text.strip()[12:]
            except:
                fecha = np.nan

        attrs_ubic= {"id": "prefijoPoblacion"}
        try:
            ubicacion = soup_oferta.find("div", class_= "row-matrioska").find("span", attrs= attrs_ubic).text.replace(",", "").strip()
        except:
            ubicacion = np.nan

        attrs_descr= {"id": "prefijoDescripcion1"}
        try:
            descripcion = soup_oferta.find("div", attrs= attrs_descr).text.replace("\n", "").strip()
        except:
            descripcion = np.nan
        try:
            lista_li = soup_oferta.find("div", class_= "row-matrioska").find_all("li")        

            for li in lista_li:

                tag = li.find("span").text[:7]

                if tag == "Salario":
                    salario = li.find("span").text[8:]

                elif tag == "Experie":
                    experiencia = li.find("span").text[20:]

                elif tag == "Tipo de":

                    tipo_contrato = li.find("span").text[18:].split(",")[0]

                    jornada = li.find("span").text[18:].split(",")[-1]
        except:
            salario = np.nan
            experiencia = np.nan
            tipo_contrato = np.nan
            jornada = np.nan

        try:
            lista_herramientas = soup_oferta.find("div", class_= "inner-expanded").find("ul", class_= "list-default").find_all("a")
            herramientas = [herramienta.text for herramienta in lista_herramientas]
        except:
            herramientas = np.nan

        try:
            funciones = soup_oferta.find("div", class_= "border-top").find("ul").find("span", class_= "list-default-text").text
        except:
            funciones = np.nan

        datos_ofertas_empleo.append([titulo, empresa, fecha, herramientas, descripcion, ubicacion, funciones, jornada, experiencia, tipo_contrato, salario, fecha_scrapeo, empleo, portal])
        contador += 1

        if contador % 100 == 0:
            sleep(20)
            
    df_infojobs = pd.DataFrame(datos_ofertas_empleo, columns= ["titulo", "empresa", "fecha", "herramientas", "descripcion", "ubicacion", "funciones", "jornada", "experiencia", "tipo_contrato", "salario", "fecha_scrapeo", "url", "portal"])
    df_infojobs = df_infojobs.dropna(how= "all")
    
    return df_infojobs